In [1]:
from mcli import predict
import pandas as pd
import json
import re

In [2]:
## convert data label into list of list
# test_data = pd.read_csv("/Users/rui.chen@avalara.com/Downloads/test.tsv", sep="\t")
# test_data["output"] = test_data["output"].map(json.loads)
# test_data["output"] = test_data["output"].map(lambda x: [list(obj.values()) for obj in x])
# test_data.to_csv("test.tsv", sep="\t")

In [11]:
test_data = pd.read_csv("test.tsv", sep="\t")

finetuned_host = "https://raw-sut-finetuned-json-lajhfg.inf.hosted-on.mosaicml.hosting"
test_data

,Unnamed: 0,document_url,input,output
0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['George TBD', 'Local Sales Tax Rate', '0.019..."
1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Gig Harbor', 'Local Sales Tax Rate', '0.022..."
2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Combined Sales Tax Rate'..."
4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
...,...,...,...,...
95,95,local-tax-changes-10-01-2021-dd4828.pdf,"\n,Local Tax Changes\n,Effective October 1, 20...","[['Hankinson', 'City sales, use, and gross rec..."
96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[['Beulah', 'city sales, use, and alcohol gros..."
97,97,tb-01-21-7a5d19.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Orderville (Kane County)', 'Transient Room ..."
98,98,tb-02-22-a5b2d2.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Eagle Mountain (Utah County)', 'Transient R..."


In [12]:
failed_indices = []
predictions_finetuned = []
for i, row in test_data.iterrows():
    try:
        prefix = "Instruct: Based on the input, generate a list of json objects with each containing 5 entities: Jurisdiction, target, new_rate, effective_from, expire_date. Input: "
        input = prefix + row["input"]
        input = "### Response: " + input
        i_prediction = predict(finetuned_host, 
                                    {
                                        "inputs": [input], 
                                        "temperature": 0.,
                                        "max_length": 8192,
                                    }, 
                                    timeout=2000)
        predictions_finetuned.append(i_prediction["output"][0])
        print(i)
    except:
        print(f"{i} failed")
        failed_indices.append("[[]]")





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [13]:
import pickle

with open('raw-prediction.pkl', 'wb') as f:
    pickle.dump(predictions_finetuned, f)

In [5]:
def get_labels(i, text):
    # Search for content including the [[...]]
    # match = re.search(r'(\[\[.*?\]\])', text)
    
    # if match:
    #     content_with_brackets = match.group(1)
    #     return content_with_brackets
    # else:
    #     print("No labels found!", i)
    #     return None

    res = "[[" + text.split("[[")[-1]
    if res.endswith("]]"):
        return res
    else:
        return res + "']]"

    
predictions = [get_labels(i, item) for i, item in enumerate(predictions_finetuned)]

In [16]:
import pickle
with open('raw-prediction.pkl', 'rb') as fp:
    raw_prediction = pickle.load(fp)

with open('ava-prediction.pkl', 'rb') as fp:
    ava_prediction = pickle.load(fp)

In [30]:
def get_response(text):
    text = text.split("### Response: ")[-1]
    return text

ava_prediction = [get_response(i) for i in ava_prediction]
raw_prediction = [get_response(i) for i in raw_prediction]

In [31]:
df = pd.read_csv("/Users/rui.chen@avalara.com/Downloads/test-data-with-predictions.tsv", sep='\t')
df

,Unnamed: 0,document_url,input,output,predictions
0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""George TBD"", ""target"": ""Loc...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',..."
1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Gig Harbor"", ""target"": ""Loc...","[""[['Gig Harbor', 'Sales and use tax', '0.002'..."
2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',..."
3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Lodging Sales Tax Rate..."
4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',..."
...,...,...,...,...,...
95,95,local-tax-changes-10-01-2021-dd4828.pdf,"\n,Local Tax Changes\n,Effective October 1, 20...","[{""jurisdiction"": ""Hankinson"", ""target"": ""City...","[""[['Hankinson', 'city sales, use, and alcohol..."
96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa...","[""[['Beulah', 'city sales, use, and alcohol gr..."
97,97,tb-01-21-7a5d19.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[{""jurisdiction"": ""Orderville (Kane County)"", ...","[""[['Orderville (Kane County)', 'Transient Roo..."
98,98,tb-02-22-a5b2d2.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[{""jurisdiction"": ""Eagle Mountain (Utah County...","[""[['Eagle Mountain (Utah County)', 'transient..."


In [33]:
# test_data["prediction"] = predictions
# test_data
df["ava_prediction"] = ava_prediction
df['raw_prediction'] = raw_prediction
df

,Unnamed: 0,document_url,input,output,predictions,ava_prediction,raw_prediction
0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""George TBD"", ""target"": ""Loc...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Gig Harbor"", ""target"": ""Loc...","[""[['Gig Harbor', 'Sales and use tax', '0.002'...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Lodging Sales Tax Rate...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Grays Harbor County"", ""targ..."
4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
...,...,...,...,...,...,...,...
95,95,local-tax-changes-10-01-2021-dd4828.pdf,"\n,Local Tax Changes\n,Effective October 1, 20...","[{""jurisdiction"": ""Hankinson"", ""target"": ""City...","[""[['Hankinson', 'city sales, use, and alcohol...","[{""jurisdiction"": ""ankinson"", ""target"": ""city ...","[{""jurisdiction"": ""Hankinson"", ""target"": ""city..."
96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa...","[""[['Beulah', 'city sales, use, and alcohol gr...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa..."
97,97,tb-01-21-7a5d19.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[{""jurisdiction"": ""Orderville (Kane County)"", ...","[""[['Orderville (Kane County)', 'Transient Roo...","[{""jurisdiction"": ""Orderville (Kane County)"", ...","[{""jurisdiction"": ""Orderville (Kane County)"", ..."
98,98,tb-02-22-a5b2d2.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[{""jurisdiction"": ""Eagle Mountain (Utah County...","[""[['Eagle Mountain (Utah County)', 'transient...","[{""jurisdiction"": ""Rich County"", ""target"": ""Tr...","[{""jurisdiction"": ""Rich County"", ""target"": ""Tr..."


In [36]:
df.to_csv("mpt30b_predictions.tsv", sep="\t")


In [9]:
# predict(finetuned_host, 
#                                     {
#                                         "inputs": ["Hello"], 
#                                         "temperature": 0.,
#                                         "max_length": 8192,
#                                     }, 
#                                     timeout=600)

In [59]:
df = pd.read_csv("/Users/rui.chen@avalara.com/Downloads/mpt30b_predictions.tsv", sep="\t")
df



,Unnamed: 0.1,Unnamed: 0,document_url,input,output,predictions,ava_prediction,raw_prediction
0,0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""George TBD"", ""target"": ""Loc...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
1,1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Gig Harbor"", ""target"": ""Loc...","[""[['Gig Harbor', 'Sales and use tax', '0.002'...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
2,2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
3,3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Lodging Sales Tax Rate...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Grays Harbor County"", ""targ..."
4,4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[""[['Unincorp. Areas', 'Local Sales Tax Rate',...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":...","[{""jurisdiction"": ""Unincorp. Areas"", ""target"":..."
...,...,...,...,...,...,...,...,...
95,95,95,local-tax-changes-10-01-2021-dd4828.pdf,",Local Tax Changes\n,Effective October 1, 2021...","[{""jurisdiction"": ""Hankinson"", ""target"": ""City...","[""[['Hankinson', 'city sales, use, and alcohol...","[{""jurisdiction"": ""ankinson"", ""target"": ""city ...","[{""jurisdiction"": ""Hankinson"", ""target"": ""city..."
96,96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa...","[""[['Beulah', 'city sales, use, and alcohol gr...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa...","[{""jurisdiction"": ""Beulah"", ""target"": ""city sa..."
97,97,97,tb-01-21-7a5d19.pdf,",Changes in Utah laws or Tax\n,Commission rule...","[{""jurisdiction"": ""Orderville (Kane County)"", ...","[""[['Orderville (Kane County)', 'Transient Roo...","[{""jurisdiction"": ""Orderville (Kane County)"", ...","[{""jurisdiction"": ""Orderville (Kane County)"", ..."
98,98,98,tb-02-22-a5b2d2.pdf,",Changes in Utah laws or Tax\n,Commission rule...","[{""jurisdiction"": ""Eagle Mountain (Utah County...","[""[['Eagle Mountain (Utah County)', 'transient...","[{""jurisdiction"": ""Rich County"", ""target"": ""Tr...","[{""jurisdiction"": ""Rich County"", ""target"": ""Tr..."


In [60]:
for i, elem in enumerate(df["ava_prediction"].tolist()):
    print(i)
    a = json.loads(elem)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [61]:
df["ava_prediction"][13]

'[{"jurisdiction": "Unincorp. Areas", "target": "Local Sales Tax Rate", "new_rate": "0.029", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Areas", "target": "State Sales Tax Rate", "new_rate": "0.065", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Areas", "target": "Total Sales Tax Rate", "new_rate": "0.094", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Non-RTA", "target": "Local Sales Tax Rate", "new_rate": "0.015", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Non-RTA", "target": "State Sales Tax Rate", "new_rate": "0.065", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Non-RTA", "target": "Total Sales Tax Rate", "new_rate": "0.08", "expire_date": "nan", "effective_from": "2021-07-01 00:00:00"}, {"jurisdiction": "Unincorp. Non-RTA HBZ", "target": "Local Sale